__Compare acoustic FDFD solution with analytical solution__

Daniel Köhn 
Kiel, 21/09/2016

__Import Libraries__

In [1]:
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
from matplotlib.colors import LightSource, Normalize
from matplotlib.pyplot import gca
from pylab import rcParams
from matplotlib import rc
from copy import copy
import scipy.special as sp
from mpl_toolkits.axes_grid1 import make_axes_locatable
import pickle

__FD grid dimensions__

In [2]:
DH = 40.0;
NX = 250;
NY = 125;

__Define fonts__

In [3]:
FSize = 20
font = {'color':  'black',
        'weight': 'normal',
        'size': FSize,
        'family': 'serif'} 
rcParams['figure.figsize'] = 10, 8

__Read true P-wave velocity model__

In [4]:
f1 = open ('snap/incl2_shot_1.p')
data_type = np.dtype ('float32').newbyteorder ('<')
p = np.fromfile (f1, dtype=data_type)
p = p.reshape(NX,NY)
p = np.transpose(p)
p = np.flipud(p)
p = p / np.amax(p)

__Define Axis__

In [5]:
x = np.arange(DH, DH*NX, DH)
y = np.arange(DH, DH*NY, DH)
lim = min(x),max(x),min(y),max(y);

__Define Acquisition geometry__

Define source positions

In [6]:
XSRC = 500.0
YSRC = 400.0

Calculate source-receiver distance r 

In [7]:
xv, yv = np.meshgrid(x, y)
r = np.sqrt((xv-XSRC)**2+(yv-YSRC)**2)

Define discrete frequencies

In [8]:
f = 20.0
w = 2.0 * np.pi * f

**Calculate analytical solution for homogeneous acoustic full space at each receiver position**

In [9]:
V = 4000.0
p_analy = -np.flipud(np.real(0-1j * np.pi * sp.hankel2(0,(r*w/V))))
p_analy = p_analy / np.amax(p_analy)
vmin = -1.0
vmax = 1.0

__Define SubPlot__

In [10]:
def do_plot(n, cm, im, title):
    
    ax=plt.subplot(2, 1, n)
    
    # define tick marks
    #ax.set_xticks([12000, 13000, 14000, 15000, 16000])
    #ax.set_yticks([0, 100, 200, 300, 400])
    
    if n==1:
        ax.set_xticks([ ])
    
    # testing some stuff 
    #plt.rc('font',**{'family':'sans-serif','sans-serif':['Helvetica']})
    ## for Palatino and other serif fonts use:
    #rc('font',**{'family':'serif','serif':['Palatino']})
    #plt.rc('text', usetex=True)
    # plt.pcolor(x, y, vp, cmap=cm, vmin=vpmin)
    
    # imshow of Vp models
    cs = plt.imshow(im,  cmap=cm, interpolation='none', extent=[min(x),max(x),min(y),max(y)], vmin=vmin, vmax=vmax)
    
    # plot source and receiver positions
    plt.plot(XSRC,YSRC,'r*',linewidth=3,markersize=15);
        
    # display values below lower clip value vmin as white 
    cs.cmap.set_under('w')
    
    # limit imshow 
    plt.axis(lim);
    
    # apply font properties to tick labels
    a = gca()
    a.set_xticklabels(a.get_xticks(), font)
    a.set_yticklabels(a.get_yticks(), font)
    
    # scale axis
    #plt.axis('scaled') 
    #ax.set_aspect('auto')
    
    # label title and axis and apply font properties
    plt.title(title, fontdict=font)
    plt.ylabel('Depth [m]', fontdict=font)
    if n==2:
        plt.xlabel('x [m]', fontdict=font)
    plt.gca().invert_yaxis()
    
    # define colorbar
    
    # fit and label colorbar\n",
    divider = make_axes_locatable(ax)
    cax = divider.append_axes("right", size="5%", pad=0.05)
    cbar = plt.colorbar(cs, cax=cax)
    cbar.set_label("Real(p)", fontdict=font, labelpad=1)
    cbar.ax.tick_params(labelsize=FSize) 

__Plot SubPlots__

In [11]:
plt.close('all')
plt.figure()
do_plot(1,'seismic', p, 'GERMAINE')
do_plot(2,'seismic', p_analy, 'analytical solution')

# optimize layout
plt.tight_layout()
#plt.savefig('test.png', format='png', dpi=100)
plt.savefig('GERMAINE_analy_comp.pdf', format='pdf')
plt.show()

In [12]:
plt.plot(p_analy[50][:],'r-')
plt.plot(p[50][:],'b-')
plt.savefig('GERMAINE_analy_comp_profile.pdf', format='pdf')
plt.show()